# The Weather Company PixieApp

## This notebook shows you how to:
- Use the Weather Company Data API to get weather forecast json data based on latitude and longitude
- Convert this json data into a pandas DataFrame
- Create a weather dashboard

## Load and install packages
First, uncomment the lines in the below cell and upgrade the pixiedust and bokeh packages. When this is done restart the kernel. You have to do this only once, or when there is an update available.

Then import the packages needed to run this notebook.

In [ ]:
!pip install --upgrade pixiedust
!pip install --upgrade bokeh
!pip install pandas
!pip install numpy

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
import time
import pixiedust

## Get The Weather Company API info

If you have an IBM Bluemix account, the info to get the Weather Company API credentials can be found within the [Weather Company Data service](https://console.bluemix.net/docs/services/Weather/index.html). In the Service Credentials tab, click New Credential, Add, and then click "view credentials" under the Actions column.

In [ ]:
# @hidden_cell
# Weather company data API credentials
username = 'YOUR-USERNAME-HERE'
password = 'YOUR-PASSWORD-HERE'
host = "twcservice.mybluemix.net"

Creating an array of locations

In [ ]:
places = sorted([
    ('Boston, MA',42.350390, -71.166406),
    ('Philidelphia, PA', 39.950296, -75.166624),
    ('Pittsburgh, PA', 40.440625, -79.995886),
    ('Hartford, CT', 41.763711, -72.685093 ),
    ('New Haven, CT', 41.308274, -72.927884),
    ('New York City, NY', 40.712852, -74.004103),
    ('Syracuse, NY', 43.048402, -76.146319),
    ('Binghamton, NY', 42.098687, -75.917974),
    ('Ithaca, NY', 42.443961, -76.501881),
    ('Beacon, NY', 41.504816, -73.969583)], key=lambda tup: tup[1])

## Convert json data to Pandas DataFrames for multiple locations

- forecast_df features info for each time for each location.
- current_df is for the most recent time - which is also the first row - for each location.

In [ ]:
forecast_df = pd.DataFrame()
current_df = pd.DataFrame()

for place in places:
    lat = place[1]
    lon = place[2]
    url = "https://{}:{}@{}/api/weather/v1/geocode/{}/{}/forecast/intraday/10day.json".format(username, password, str(host), str(lat), (lon))                                                                                                       
    res = requests.get(url)
    weather = json.loads(res.text)
    # create dataframe
    weatherdf = pd.DataFrame.from_dict(weather['forecasts'][0], orient='index').transpose()
    # get all forecasts
    for forecast in weather['forecasts'][1:]:
        weatherdf = pd.concat([weatherdf, pd.DataFrame.from_dict(forecast,orient='index').transpose()])
    # add date column
    weatherdf['date'] = weatherdf['fcst_valid_local'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S-0400'))
    # get rid of unneeded columns
    weatherdf = weatherdf.drop(['expire_time_gmt','num','qualifier','qualifier_code'],1)   
    weatherdf = weatherdf.drop(['fcst_valid','fcst_valid_local','icon_extd','wdir_cardinal'],1)   
    weatherdf = weatherdf.drop(['subphrase_pt1','subphrase_pt2','subphrase_pt3','class'],1)   
    weatherdf = weatherdf.drop(['daypart_name','phrase_12char','phrase_22char','phrase_32char'],1)   
    # rename columns
    weatherdf.columns = ['pop','wind_speed','relative_humidity','icon_code','clouds','wind_direction','temperature', 'precipitation_type', 'day_of_week', 'date']
    weatherdf[['pop','wind_speed','relative_humidity','clouds','wind_direction','temperature']] = weatherdf[['pop','wind_speed','relative_humidity','clouds','wind_direction','temperature']].apply(pd.to_numeric)
    # copy pop column into new rain column
    weatherdf['rain'] = weatherdf['pop'].as_matrix()
    # get rid of pop column
    weatherdf = weatherdf.drop('pop',1)
    # make place, lat, and lon columns
    weatherdf['lat'] = lat
    weatherdf['lon'] = lon
    weatherdf['place'] = place[0]
    # append info to data frames
    forecast_df = forecast_df.append(weatherdf)
    current_df = current_df.append(weatherdf.iloc[0])

current_df.reset_index(inplace=True)
forecast_df.reset_index(inplace=True)
current_df
# forecast_df

This is getting the icon code to display an image associated with the most recent upcoming weather in Boston.

In [ ]:
BostonDF = forecast_df.loc[forecast_df["place"]=="Boston, MA"]
icon= BostonDF.iloc[0]['icon_code']
icon

BostonDF will also be used as the default chart to be shown when choosing a weather forecast variable.

## Creating the PixieApp

In [ ]:
weatherAppHTML = """
    <!-- Title bar -->
    <nav class="navbar navbar-default">
        <div class="container-fluid">
            <div class="navbar-header">
                <a class="navbar-brand">WEATHER IN NEW ENGLAND</a>
            </div>
            <div class="nav navbar-nav navbar-right">
                <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/IBM_logo.svg/1000px-IBM_logo.svg.png" alt="IBM logo" width="50" height="25">
            </div>
        </div>
    </nav>

    <!-- Description -->
    <div class="jumbotron container-fluid">
        <div class="col-sm-3">
            <img src="{{this.iconURL}}" id="forecast-icon" class="img-responsive center-block"/>
        </div>
        <div class="col-sm-9">
            <p>This is a PixieApp utilizing The Weather Company API.
            The mapbox map on the left shows the temperature, rain, wind speed, wind direction, and relative humidity values for various locations
            in the New England area for the most recent time gathered from The Weather Company. The chart on the right shows different weather
            charts for Boston.</p>
        </div>
    </div>

    <!-- Main content -->
    <div class="container">
        <div class="row">
            <div class="col-sm-6">
                <!-- Dropdown buttons -->
                <div id="dropdown-container">
                    <div class="chart-dropdown col-sm-12">
                        <button class="col-sm-12 btn btn-primary btn-lg">Choose a weather variable to display</button>
                        <div class="dropdown-content col-sm-12">
                            {% for info in this.measures %}
                                <a pd_script="self.currentChartInfo={{info}}">{{info["label"]}}
                                    <target pd_target="title{{prefix}}" pd_script="print('{} for Boston, MA'.format('{{info['label']}}'))"/>
                                    <target pd_target="map{{prefix}}"
                                            pd_entity="pixieapp_entity[1]"
                                            pd_options="handlerId=mapView;preserveCols=place;rendererId=mapbox;keyFields=lat,lon;valueFields={{info["valueField"]}};opacity=100;kind=choropleth;mapboxtoken=pk.eyJ1Ijoiam9yZGFuZ2VvcmdlIiwiYSI6ImNqM2s3YjhlcjAwamgzMm4zeWMwOWo3cjMifQ.AQqZdo1kVVjKSIsqqdEGRQ"/>
                                    <target pd_target="charts{{prefix}}"
                                            pd_entity="chartDF"
                                            pd_options="rendererId=bokeh;keyFields=date;aggregation=AVG;handlerId=lineChart;valueFields={{info["valueField"]}};rowCount=10000;legend=false;chartsize=100"/>
                                </a>
                            {% endfor %}
                        </div>
                    </div>
                </div>

                <!-- Map display -->
                <div class="col-sm-12" id="map{{prefix}}">
                </div>
            </div>

            <div class="col-sm-6">
                <center class="col-sm-12"><h1 id="title{{prefix}}"/></center>
                <div class="col-sm-12 chartDisplay" id="charts{{prefix}}"></div>
            </div>

            <pd_event_handler pd_origin="map{{prefix}}">
                <target pd_target="title{{prefix}}" pd_script="print('{} for {}'.format(self.currentChartInfo['label'], eventInfo['place']))"/>
                <target pd_target="charts{{prefix}}"
                        pd_script="self.loadCityDF(eventInfo['place'])"
                        pd_options="chart=true" pd_refresh/>
            </pd_event_handler>
        </div>
    </div>
"""

In [ ]:
weatherAppCSS = """
    .chartDisplay {
        margin-top: 20px;
    }
    
    .navbar-brand {
        font-size: 20px;
        letter-spacing: 30px;
    }
    
    #dropdown-container {
        margin-bottom: 60px;
    }
    
    #forecast-icon {
        position: relative;
        top: 25%;
        left: 25%;
        width: 75px;
        height: 75px;
    }

    /* The container <div> - needed to position the dropdown content */
    .chart-dropdown {
        position: absolute;
        display: inline-block;
        width: 100%;
    }
    /* Dropdown Content (Hidden by Default) */
    .dropdown-content {
        display: none;
        position: absolute;
        background-color: #f9f9f9;
        box-shadow: 0px 8px 16px 0px rgba(0,0,0,0.2);
        overflow: visible;
        z-index: 1000000;
    }
    /* Links inside the dropdown */
    .dropdown-content a {
        color: black;
        padding: 12px 16px;
        text-decoration: none;
        display: block;
    }
    /* Change color of dropdown links on hover */
    .dropdown-content a:hover {
        background-color: #f1f1f1;
    }
    /* Show the dropdown menu on hover */
    .chart-dropdown:hover .dropdown-content {
        display: block;
    }
"""

In [ ]:
from pixiedust.display.app import *

@PixieApp
class WeatherApp:
    def setup(self):
        self.iconURL = 'https://github.com/ibm-cds-labs/python-notebooks/blob/master/weathericons/icon'+str(icon)+'.png?raw=true'
        self.measures = [
            {"label": "Temperature (F)", "valueField": "temperature"},
            {"label": "Rain (%)", "valueField": "rain"},
            {"label": "Wind Speed (mph)", "valueField": "wind_speed"},
            {"label": "Relative Humidity (%)", "valueField": "relative_humidity"},
        ]

    def loadCityDF(self, place):
        self.cityDF = forecast_df.loc[forecast_df["place"]==place]
        
    @property
    def chartDF(self):
        return self.cityDF if hasattr(self, "cityDF") else self.pixieapp_entity[0]

    @route(chart="true")
    def showChart(self):
        return """
            <div pd_entity="chartDF"
                pd_render_onload
                pd_options="debug=true;noChartCache=true;rendererId=bokeh;keyFields=date;aggregation=AVG;handlerId=lineChart;valueFields={{this.currentChartInfo["valueField"]}};rowCount=10000;legend=false;chartsize=100">
            </div>
        """

    @route()
    def main(self):
        return """<style>{}</style>{}""".format(weatherAppCSS, weatherAppHTML)

# run the app
dfs = [BostonDF, current_df]
WeatherApp().run(dfs, runInDialog='false')

The cell below can be used for debugging with/for PixieDust

In [ ]:
%pixiedustLog -l debug